#修了課題DEMO①　Titanic

##はじめに
この修了課題では、Titanic号の乗客の情報を利用して、

テストデータで用意した乗客が生き残れるかを予測します。

今回はDEMOとして、２層のニューラルネットワークを用いて

クラス(pclass)・性別(sex)・乗船港(emberked)の３つの要素を取り上げながら、

学習をさせてみましょう。

##作成までの流れ
大まかな流れとして
1. データのダウンロードと成形

   データダウンロードしてそのまま活用するのは困難です。
   モデルが学習できるよう、カテゴリ変数に置き換えたり、
   欠損値を補完したりなど、成形する必要があります。
2. モデルの構築

   学習を行うモデルのアルゴリズムを理解して、コードを作成します。

3. 学習と結果

   学習を行い結果を確認してみます。
   精度が目標まで達したら、提出用のデータを作成します。

#1.データのダウンロードと成形

## データのダウンロード

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# 学習用のデータのダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1-12Pg5IsjNAEbgk7G4a2WqFaOhpYmbyJ' -O titanic_train.csv
# 検証用のデータのダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1jmzmYNPRWUGLcHeqhcwKTGlb_d2Rzn4Y' -O titanic_validation.csv

In [ ]:
# 学習用データ
train_df = pd.read_csv('titanic_train.csv', index_col=0)
display(train_df.head(3))

In [ ]:
# 検証用データ
val_df = pd.read_csv('titanic_validation.csv', index_col=0)
display(val_df.head(3))

学習用データと検証用データの情報を確認してみましょう。

欠損値がないようにみえますが、？で埋められているため

np.nanで置き換えてあげましょう。

また、"Dtype"がint64のものは全てのデータがint64型を示していますが、

"Dtype"がobjectのものは、データの中身に複数の型が紛れています。

もし、"Dtype"がobjectになっているデータも使用したいときは

型を統一するように処理をかける必要があることに注意しましょう。


In [ ]:
train_df = train_df.replace('?', np.nan)
val_df = val_df.replace('?', np.nan)
train_df.info()
val_df.info()

## データの成形

今回使用する２層のニューラルネットワークのために必要な処理を施していきます。

In [ ]:
#学習用データと検証用データの２つを繋げて、同時に処理できるようにしておく
dataset = [train_df, val_df]

## 生存と関係性の強い項目の確認

今回取り上げる
クラス(pclass)・性別(sex)・乗船港(emberked)について

どの程度生存に関連しているかを確かめてみましょう。

In [ ]:
# クラス pclass
train_df[['pclass', 'survived']].groupby(['pclass'], as_index=False).mean().sort_values(by='survived', ascending=False)

In [ ]:
# 性別 sex
train_df[["sex", "survived"]].groupby(['sex'], as_index=False).mean().sort_values(by='survived', ascending=False)

In [ ]:
# 乗船港 emberked
train_df[["embarked", "survived"]].groupby(['embarked'], as_index=False).mean().sort_values(by='survived', ascending=False)

In [ ]:
# 上記の3変数で予測するので、１つにまとめる
columns = ['pclass','sex','embarked', ]

## 欠損値処理

ここで一度、学習用データと検証用データを確認してみましょう。

すると、検証用データの「emberked」には値が？になっているものがあります。

先ほど、データをダウンロードしたときに「val_df.replace('?', np.nan)」としているので、

その欠損値をさらに最頻値に置き換えてみます。

In [ ]:
#欠損値の部分を削除する
freq_port = train_df.embarked.dropna().mode()[0]
#空いた部分に最頻値を入れて埋める
train_df['embarked'].fillna(freq_port, inplace=True)
val_df['embarked'].fillna(freq_port, inplace=True)

## カテゴリーデータの処理

乗船港と性別は文字で登録されているので、

数値に置き換えてみましょう。

それぞれにint型でカテゴリー変数に変えてみます。

In [ ]:
for data in dataset:
  data['embarked'] = data['embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

In [ ]:
for data in dataset:
    data['sex'] = data['sex'].map( {'female': 1, 'male': 0} ).astype(int)

## データの分割
学習用データと検証用データの中にはそれぞれ

'pclass','sex','embarked','survived'が格納されています。

生存者予測を学習するので、'pclass','sex','embarked'と'survived'に分けましょう。

In [ ]:
X_train = train_df[['pclass','sex','embarked']]
y_train = train_df['survived']
X_val = val_df[['pclass','sex','embarked']]
y_val = val_df['survived']

# 2.モデルの構築

## NNの構築

損失関数、活性化関数、ニューラルネットワークを構築します。

損失関数はMSE、活性化関数はシグモイド関数、

ネットワークは２層のニューラルネットワークになるように作っていきます。

In [ ]:
# バッチ版mse
def rmse(y, t):
    delta = 1e-7
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)

    mini_batch = y.shape[0]
    t = t.astype(int)
    return np.sqrt(np.sum((y - t)**2) / mini_batch)

# 活性化関数
# シグモイド関数
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def deriv_sigmoid(x):
    return (1 - sigmoid(x)) * sigmoid(x)

In [ ]:
# 2層のニューラルネットワーク
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):

        # 重みの初期化
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        # forward
        u1 = np.dot(x, W1) + b1
        z1 = sigmoid(u1)
        u2 = np.dot(z1, W2) + b2
        y = sigmoid(u2)

        return y

    # x:入力データ, t:教師データ
    def loss(self, x, t):
        y = self.predict(x)

        return rmse(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.where(y >= 0.5, 1, 0)
        #y = np.argmax(y, axis=1)
        #t = np.argmax(t, axis=1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy

    # 数値計算で勾配を求めると計算時間がかかるため、逆伝播法をはじめから実装する
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}

        batch_num = x.shape[0]

        # forward
        u1 = np.dot(x, W1) + b1
        z1 = sigmoid(u1)
        u2 = np.dot(z1, W2) + b2
        y = sigmoid(u2)

        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)

        du1 = np.dot(dy, W2.T)
        dz1 = deriv_sigmoid(u1) * du1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

In [ ]:
# ３変数で予測するため、入力層のサイズは３
# 二値分類のため、出力層のサイズは１
network = TwoLayerNet(input_size=3, hidden_size=100, output_size=1)

次にハイパーパラメータを設定します。

エポック数や学習率は値によって学習の精度が変わりますので、

いくつか試してみましょう。

今回はDEMOとして以下の値を設定しました。

In [ ]:
epoch = 1000
train_size = X_train.shape[0]
mini_batch = 20
lr = 0.01

In [ ]:
#lossの値とacc(精度)を格納する
train_loss_list = []
train_acc_list = []
val_acc_list = []

# 1エポックあたりの繰り返し数を設定する
iter_per_epoch = max(train_size / mini_batch, 1)

#3.学習から提出まで

##学習

In [ ]:
for i in range(epoch):
    # ミニバッチの取得を行う
    batch_mask = np.random.choice(train_size, mini_batch)
    x_batch = np.array(X_train.iloc[batch_mask])
    y_batch = np.array(y_train.iloc[batch_mask])[..., np.newaxis]

    # 誤差逆伝播法によって勾配を求める
    grad = network.gradient(x_batch, y_batch)

    # パラメータの更新をする
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= lr * grad[key]

    # 学習経過を記録する
    loss = network.loss(x_batch, y_batch)
    train_loss_list.append(loss)

    # 1エポックごとに認識精度を計算する
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(np.array(X_train), np.array(y_train)[..., np.newaxis])
        val_acc = network.accuracy(np.array(X_val), np.array(y_val)[..., np.newaxis])
        train_acc_list.append(train_acc)
        val_acc_list.append(val_acc)
        print("train acc, val acc | " + str(train_acc) + ", " + str(val_acc))

In [ ]:
import matplotlib.pyplot as plt

markers = {'train': 'o', 'test': 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, val_acc_list, label='val acc', linestyle='--')
plt.xlabel("epoches")
plt.ylabel('accuracy')
plt.ylim(0, 1.0)
plt.legend(loc='lower right')
plt.show()

今回の精度は約62％になりました。このモデルの仕様では目標の精度には達していませんね。

２層のニューラルネットワークでは、思うように精度が出ないようです。

モデルをより複雑なものにすることのほかにも

エポック数を増やしてみたり、予測する要素を変更するなどの処理を行ってみると

精度が向上するかもしれません。

## （参考）提出用データの作成の仕方
参考として、今回学習した２層のニューラルネットワークモデルが

テスト用のデータを予測して、結果をcsvファイルとして出力するまでを

掲載してみました。

このcsvファイルを「修了課題提出用サイト」にアップロードすると結果を確認することができます。

In [ ]:
# 提出用のデータをダウンロード
!wget 'https://drive.google.com/uc?export=download&id=1-1KX2NQmwUOXAstOE7c2INC1rRMNABVZ' -O titanic_test_x.csv

In [ ]:
X_test = pd.read_csv('titanic_test_x.csv', index_col=0)
X_test

In [ ]:
# Demoで行った処理をテスト用のデータにも行う
X_test = X_test.replace('?', np.nan)
X_test['embarked'].fillna(freq_port, inplace=True)
X_test['embarked'] = X_test['embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)
X_test['sex'] = X_test['sex'].map( {'female': 1, 'male': 0} ).astype(int)
X_test = X_test[['pclass','sex','embarked']]

In [ ]:
# 学習させたモデルで予測する
y_pred = network.predict(X_test)
# このままだと確率値がでてしまうので、確率に基づいて最終的な判定値に変換する
y_pred = np.where(y_pred >= 0.5, 1, 0)
# pandasのDataFrame形式に変換し、CSV出力する
y_pred = pd.DataFrame(y_pred, columns=['survived'])
# csv形式で提出する
y_pred.to_csv('y_pred.csv')
y_pred

## （参考）sk-learnを利用した機械学習モデルを用いた予測

In [ ]:
# ロジスティック回帰
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
logreg.fit(X_train, y_train)

acc_log = round(logreg.score(X_train, y_train) * 100, 2)
acc_log_val = round(logreg.score(X_val, y_val) * 100, 2)
print('学習精度:', acc_log)
print('検証精度:', acc_log_val)

# 予測する際
Y_pred = logreg.predict(X_test)